# **Trusted** 

A camada trusted como a primeira representação estruturada e validada dos dados após a ingestão bruta (raw). Ela transforma arquivos heterogêneos (JSON, CSV, XML, ZIP etc.) em datasets tabulares padronizados, tipados e auditáveis, mantendo fidelidade à fonte original.

***Se o Raw é evidência, a Trusted é evidência organizada.***

Ela ainda não aplica regras de negócio complexas ou integra múltiplas fontes — isso pertence às camadas seguintes (Refined, Feature, etc.). A Trusted apenas garante que os dados:

- possuem tipos corretos
- têm schema estável
- são consistentes e reprocessáveis
- mantêm rastreabilidade completa

## *BCB SGS*

### *1. Tipo e Estrutura*
Os dados extraídos da SGS são até o momento especificamente as séries SELIC (432) e IPCA (433). Por definição esses são dados diários extraídos (para IPCA o BCB aplica um interpolação para retornar o variação percentual do dia) cujo o principal interesse é *data* e *valor*. Afim de preservar uma estrutura auditável também serão gerados campos que indiquem:

- Data de Processamento (injestion)
- Arquivo Raw que gerou aquele dados
- Hash do conteúdo para governança, pois:
    - Auditável permitindo reconstruir histórico, provar integridade e fazer reconciliação
    - Caso reprocesse o mesmo período os dados não são reinseridos (indepotência)
    - Detectar alteração silenciosa (Se houver alteração de dados antigos a coluna vai acusar)
    - Unicidade Real; Pequenas variações de campo e formatação (como tipo) não impactam a base

tornando assim os dados *raw* uma *tabela clean e auditável". Com isso em mente podemos montar o schema desses dados como:

**Raw:**
 data | valor |
|:------------:|:-------:|
| str | str |

**Trusted**

| series_id | ref_date | value | raw_file | raw_hash | record_hash | ijestioningestion_ts_utc |
|:---------:|:--------:|:-----:|:--------:|:--------:|:-----------:|:------------------------:|
| int | date | float | str | str | str | str |

series_id e ref_date fornecem naturalmente uma chave única para a base de dados.

#### *Classe Modelo*

Como estamos falando de um processo fixo de injestão de dados que serão transformados em arquivos *.parquet* é interessenate construir uma classe fixa de formatação de dados não dependendo de pandas. Esse tipo de arquitetura é interessante inclusive pela restrição de flexbilidade do objeto gerando flexibilidade de código, já que seu futuramente pandas não for mais uma opção o domínio SgsPoint continua o mesmo, com controle de tipo e testabilidade.

```python models.py
@dataclass(frozen=True)
class SgsPoint:
    series_id: int
    ref_date: date
    value: Optional[float]  # pode ser None se vier vazio
    raw_file: str
    raw_hash: str
    record_hash: str
    ingestion_ts_utc: str 
```
Da mesma forma, afim de manter a escalabilidade da Base de Dados podemos já esquematizar os Metadados da séries extraídas, gerando assim maior governança dos dados e facilidade de entendimente para futuros consumidores dessa informação, tendo assim noções de fonte, nome da série, frequência de publicação, unidade sem precisar consultar documentação.

```python models.py
@dataclass(frozen=True)
class SgsSeriesMeta:
    series_id: int
    name: str
    frequency: str
    unit: str
    source: str = "BCB_SGS" # Por padrão por enquanto
```

Apartir da forma que os arquivos JSON estão sendo salvos podemos facilmente extrair de qual série são aqueles dados.

In [26]:
from pathlib import Path
import json

path = Path("C:\\Users\\Dell\\OneDrive\\Documentos\\GitHub\\ML-ETTJ26\\data\\01_raw\\bcb\\sgs\\433_01-01-2000_31-12-2008.json")

stem = path.stem
series_str = stem.split("_", 1)[0]
series_str

'433'


Com uma rápida olhada nos dados podemos perceber que as informações extaráidas da API está em uma lista de discionários onde ambas as chaves e valores são strings.

```JSON
[{'data': '01/01/2000', 'valor': '1.00'}, ...]
```


In [28]:
with path.open("r", encoding="utf-8") as f:
    json_f = json.load(f)
json_f[0]

{'data': '01/01/2000', 'valor': '0.62'}

Antes de passar esses valores para trusted eles precisam então ser normalizados:
- Data deve ser formato date, não string
- Valor deve ser valor decimal com quantidade fixa de casas, não string 

(obs: float pode gerar comportamento indesejado em razão da base binária)

In [29]:
from datetime import datetime

s = json_f[0]["data"]
datetime.strptime(s, "%d/%m/%Y").date()

datetime.date(2000, 1, 1)

In [30]:
from decimal import Decimal, ROUND_HALF_UP

s = json_f[0].get("valor")
s = str(s).strip()
s = Decimal(s)
vq = s.quantize(Decimal("0.0000000001"), rounding=ROUND_HALF_UP)
print(vq, s)

0.6200000000 0.62


Tranquilamente conseguimos Gerar agora os valores hash com a segurança de comportamento maior

In [31]:
import hashlib
series_id = int(series_str)
ref_date = datetime.strptime(json_f[0]["data"], "%d/%m/%Y").date()
value_dec = vq

payload = f"{series_id}|{ref_date.isoformat()}|{value_dec}"

record_hash = hashlib.sha256(payload.encode("utf-8")).hexdigest()
record_hash

'072692bcfd6ededdac1377bf9cab985900fca426ef5d8543c6d9a7cdc779fb24'

In [ ]:
h = hashlib.sha256()
with path.open("rb") as f:
    
    for chunk in iter(lambda: f.read(1024 * 1024), b""):
        h.update(chunk)
h.hexdigest()

'2064cccd061cb5ed9f8747e42c3cbbea8637b9542f78a6146547c9c8674f67ef'

In [33]:
from datetime import timezone

ingestion_ts_utc = datetime.now(timezone.utc).replace(microsecond=0).isoformat()
ingestion_ts_utc

'2026-02-16T21:29:27+00:00'

In [ ]:
# Após rodar o pipeline da trusted ( kedro run --pipeline trusted_bcb_sgs ) podemos ler o parquet que deve ser gerado
import pandas as pd

sgs_p = pd.read_parquet(r"C:\Users\Dell\OneDrive\Documentos\GitHub\ML-ETTJ26\data\02_trusted\bcb\sgs\points.parquet")
print(sgs_p.info(), sgs_p["raw_hash"].value_counts(), sep="\n\n")

<class 'pandas.DataFrame'>
RangeIndex: 9852 entries, 0 to 9851
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   series_id         9852 non-null   int64  
 1   ref_date          9852 non-null   object 
 2   value             9852 non-null   float64
 3   record_hash       9852 non-null   str    
 4   raw_file          9852 non-null   str    
 5   raw_hash          9852 non-null   str    
 6   ingestion_ts_utc  9852 non-null   str    
dtypes: float64(1), int64(1), object(1), str(4)
memory usage: 2.2+ MB
None

raw_hash
480c424330ea60ca1d9e880bb515e8be38bb0b61f0c3d6f6b07d640e8884bad9    3329
023c3e8e15126a84ae1b8a9e00a325719c03eb7c8f8bbb7bf1988a81415c7926    3288
74ec4cb5fdcd6e61a207f15f04df8b708d760c50f50e8cb4a1bd6538b7db72c6    2922
2064cccd061cb5ed9f8747e42c3cbbea8637b9542f78a6146547c9c8674f67ef     108
93a1f52671c819997afbd5e454898679dd21e844a227298538b613a37dae725e     108
7a8c38a1e810ad0d8478c2962c395b58

In [70]:
sgs_m = pd.read_parquet(r"C:\Users\Dell\OneDrive\Documentos\GitHub\ML-ETTJ26\data\02_trusted\bcb\sgs\series_meta.parquet")
print(sgs_m.info(), sgs_m.head(), sep="\n\n")

<class 'pandas.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   series_id    2 non-null      int64
 1   series_name  2 non-null      str  
 2   frequency    2 non-null      str  
 3   unit         2 non-null      str  
 4   source       2 non-null      str  
dtypes: int64(1), str(4)
memory usage: 244.0 bytes
None

   series_id series_name frequency    unit   source
0        432       SELIC         D  % a.a.  BCB_SGS
1        433        IPCA         M       %  BCB_SGS


## *BCB DEMAB*

Ao contrario do Sistema Gerenciador de Séries Temporais (SGS) do BCB o Departamento de Operações do Mercado Aberto (DEMAB) oferece os dados mensais de definitivos de Títulos Públicos Federais registrados no Sistema Especial de Liquidação e Custódia (o Selic), com detalhamento diário por título e vencimento,  ou seja, o exato insumo nescessário para construção da ETTJ PRE desse projeto.

Obs: Considerarei apenas os negociados Extragrupo por considerar que as informações Intragrupo (NegT) pode gerar uma espécie de ruído de preço e liquidez desnecessário.

In [7]:
from pathlib import Path
from collections import Counter
import zipfile
from datetime import datetime

zip_path = [Path(r"C:\Users\Dell\OneDrive\Documentos\GitHub\ML-ETTJ26\data\01_raw\bcb\demab\negociacoes_titulos_federais_secundario\NegE200701.ZIP"), 
            Path(r"C:\Users\Dell\OneDrive\Documentos\GitHub\ML-ETTJ26\data\01_raw\bcb\demab\negociacoes_titulos_federais_secundario\NegE202309.ZIP"),  
            Path(r"C:\Users\Dell\OneDrive\Documentos\GitHub\ML-ETTJ26\data\01_raw\bcb\demab\negociacoes_titulos_federais_secundario\NegE202512.ZIP"),
            ]

for zp in zip_path:
    with zipfile.ZipFile(zp, "r") as z:
        infos = z.infolist()
        print(f"ZIP: {zp.name}")
        exts = [Path(i.filename).suffix.lower() for i in z.infolist() if not i.is_dir()]
        print(Counter(exts))
        print(f"Arquivos: {len(infos)}")
        for i in infos[:50]:
            dt = datetime(*i.date_time)
            print(f"- {i.filename} | {i.file_size/1024:.1f} KB | {dt}")

ZIP: NegE200701.ZIP
Counter({'.csv': 1})
Arquivos: 1
- NegE200701.CSV | 94.2 KB | 2007-02-03 00:04:04
ZIP: NegE202309.ZIP
Counter({'.csv': 1})
Arquivos: 1
- NegE202309.CSV | 142.7 KB | 2023-10-03 23:47:42
ZIP: NegE202512.ZIP
Counter({'.csv': 1})
Arquivos: 1
- NegE202512.CSV | 166.5 KB | 2026-01-05 23:04:18


Rapidamente podemos conferir os arquivos em diferentes momentos do tempo explorando e garantido se há consistência na forma que os arquivos foram extraídos e como podemos ver, além do nome *NegEyyymm.ZIP* o arquivo compactado parece se manter o mesmo ao longo do tempo ( .csv ) sendo sempre único

In [10]:
for zp in zip_path:
    with zipfile.ZipFile(zp, "r") as z:
        # escolha um arquivo que pareça dados
        name = [i.filename for i in z.infolist() if i.filename.lower().endswith((".csv", ".txt"))][0]
        print("Arquivo:", name)
        with z.open(name) as f:
            sample = f.read(4096)  # 4KB
        # tente decodificar
        for enc in ("utf-8", "latin-1", "cp1252"):
            try:
                text = sample.decode(enc)
                print("Encoding provável:", enc)
                print(text.splitlines()[:10])
                break
            except UnicodeDecodeError:
                pass


Arquivo: NegE200701.CSV
Encoding provável: utf-8
['DATA MOV;SIGLA;CODIGO;CODIGO ISIN;EMISSAO;VENCIMENTO;NUM DE OPER;QUANT NEGOCIADA;VALOR NEGOCIADO;PU MIN;PU MED;PU MAX;PU LASTRO;VALOR PAR;TAXA MIN;TAXA MED;TAXA MAX', '02/01/2007;LFT;210100;BRSTNCLF1741;04/01/2002;17/01/2007;28;4223;;2963,02918000;2963,02918000;2963,02918000;2962,89340793;2963,00978687;-0,0150;-0,0150;-0,0150', '02/01/2007;LFT;210100;BRSTNCLF17W8;19/09/2002;21/02/2007;2;1259;;2963,06973000;2963,07607744;2963,07694900;2962,65014685;2963,00978687;-0,0168;-0,0166;-0,0150', '02/01/2007;LFT;210100;BRSTNCLF17X6;19/09/2002;21/03/2007;43;14756;;2963,00978600;2963,09021875;2963,09125200;2962,37521180;2963,00978687;-0,0128;-0,0127;0,0000', '02/01/2007;LFT;210100;BRSTNCLF1808;19/09/2002;20/06/2007;11;12764;;2963,00978600;2963,15668471;2963,17210800;2961,64682719;2963,00978687;-0,0119;-0,0108;0,0000', '02/01/2007;LFT;210100;BRSTNCLF1832;19/09/2002;19/09/2007;4;1113;;2963,23409641;2963,23620561;2963,23646600;2960,89513416;2963,0097

A partir daí já dá para começar a montar a trusted, pois temos que algumas colunas são fixas e tendo nas datas mais recentes com colunas a mais, irrelevantes para o modelo, mas interessante se atentar na hora de modelar o fluxo para a trusted. Assim como com os dados de SGS podemos já estruturar os dados:

**Raw:** Temos muitas colunas que vão apenas ocupar espaço e armazenar informação que nunca será usada para construção de curvas, por mais que seja interessante para analise de liquidez, por exemplo, por enquanto são desnescessárias e não serão carregadas na trusted.
| DATA MOV | SIGLA | CODIGO | CODIGO ISIN | EMISSAO | VENCIMENTO | NUM DE OPER | QUANT NEGOCIADA |
|:--------:|:-----:|:------:|:-----------:|:-------:|:----------:|:-----------:|:---------------:|

| PU MIN | PU MED | PU MAX | PU LASTRO | VALOR PAR | TAXA MIN | TAXA MED | TAXA MAX | OPER COM CORRETAGEM | QUANT NEG COM CORRETAGEM |
| :------:|:------:|:------:|:---------:|:---------:|:--------:|:-------:|:--------:|:-------------------:|:------------------------:|

**Trusted:**
| DATA MOV | SIGLA | CODIGO ISIN | EMISSAO | VENCIMENTO | PU MIN | PU MED | PU MAX | PU LASTRO | VALOR PAR | TAXA MIN | TAXA MED | TAXA MAX |
|:--------:|:-----:|:-----------:|:-------:|:----------:|:------:|:------:|:------:|:---------:|:---------:|:--------:|:--------:|:--------:|

Já aqui podemos podemos fazer a separação por fato e dimensão e construir os modelos de domínio

```python models.py
@dataclass(frozen=True)
class DemabQuoteDaily:
    trade_date: date
    codigo_isin: str

    pu_min: Optional[float]
    pu_med: Optional[float]
    pu_max: Optional[float]
    pu_lastro: Optional[float] 
    valor_par : Optional[float]

    taxa_min: Optional[float]
    taxa_med: Optional[float]
    taxa_max: Optional[float]

    ref_month: str
    raw_zip_file: str
    raw_zip_hash: str
    inner_file: str
    record_hash: str
    ingestion_ts_utc: str 


@dataclass(frozen=True)
class DemabInstrument:
    codigo_isin: str
    sigla: str
    emissao_date: date
    vencimento_date: date
    source: str = "BCB_DEMAB" # Por padrão por enquanto
```
onde record_hash será montada por : isin | trade_date | pu_med | taxa_med


In [2]:
import pandas as pd
df = pd.read_parquet(r"C:\Users\Dell\OneDrive\Documentos\GitHub\ML-ETTJ26\data\02_trusted\bcb\demab\quotes_daily\2008-08.parquet")
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 746 entries, 0 to 745
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   trade_date        746 non-null    object 
 1   isin              746 non-null    str    
 2   pu_min            658 non-null    float64
 3   pu_med            658 non-null    float64
 4   pu_max            658 non-null    float64
 5   pu_lastro         746 non-null    float64
 6   valor_par         746 non-null    float64
 7   taxa_min          425 non-null    float64
 8   taxa_med          425 non-null    float64
 9   taxa_max          425 non-null    float64
 10  raw_zip_file      746 non-null    str    
 11  raw_zip_hash      746 non-null    str    
 12  inner_file        746 non-null    str    
 13  record_hash       746 non-null    str    
 14  ingestion_ts_utc  746 non-null    str    
dtypes: float64(8), object(1), str(6)
memory usage: 228.2+ KB


In [5]:
df[df["pu_med"].isna() & df["taxa_med"].isna()]["trade_date"].unique()

array([datetime.date(2008, 8, 1), datetime.date(2008, 8, 4),
       datetime.date(2008, 8, 5), datetime.date(2008, 8, 6),
       datetime.date(2008, 8, 7), datetime.date(2008, 8, 8),
       datetime.date(2008, 8, 11), datetime.date(2008, 8, 12),
       datetime.date(2008, 8, 13), datetime.date(2008, 8, 14),
       datetime.date(2008, 8, 15), datetime.date(2008, 8, 18),
       datetime.date(2008, 8, 19), datetime.date(2008, 8, 20),
       datetime.date(2008, 8, 21), datetime.date(2008, 8, 22),
       datetime.date(2008, 8, 25), datetime.date(2008, 8, 26),
       datetime.date(2008, 8, 27), datetime.date(2008, 8, 28),
       datetime.date(2008, 8, 29)], dtype=object)

## *B3 Price Report*
Assim como no DEMAB os arquvios B3 são zipados, porém agora cada arquivo representa a extração de um dia. Price Report especificamente contém o relatório completo detalhado por dia do pregão. A decisão de usar esse price report para capturar as negociações de futuro de DI ao invés do simplificado é por conta quantidade de dados históricos disponíveis, já que apenas recentemente começaram a separar o mercado de ações e derivativos em dois reports simplificados distintos.



In [1]:
from pathlib import Path
from collections import Counter
import zipfile
from datetime import datetime

zip_path = [Path(r"C:\Users\Dell\OneDrive\Documentos\GitHub\ML-ETTJ26\data\01_raw\b3\PriceReport\PR200102_20200102.zip"), 
            Path(r"C:\Users\Dell\OneDrive\Documentos\GitHub\ML-ETTJ26\data\01_raw\b3\PriceReport\PR220517_20220517.zip"),  
            Path(r"C:\Users\Dell\OneDrive\Documentos\GitHub\ML-ETTJ26\data\01_raw\b3\PriceReport\PR260204_20260204.zip"),
            ]

for zp in zip_path:
    with zipfile.ZipFile(zp, "r") as z:
        infos = z.infolist()
        print(f"ZIP: {zp.name}")
        exts = [Path(i.filename).suffix.lower() for i in z.infolist() if not i.is_dir()]
        print(Counter(exts))
        print(f"Arquivos: {len(infos)}")
        for i in infos[:50]:
            dt = datetime(*i.date_time)
            print(f"- {i.filename} | {i.file_size/1024:.1f} KB | {dt}")

ZIP: PR200102_20200102.zip
Counter({'.zip': 1})
Arquivos: 1
- PR200102.zip | 2417.4 KB | 2026-02-15 23:40:14
ZIP: PR220517_20220517.zip
Counter({'.zip': 1})
Arquivos: 1
- PR220517.zip | 4915.4 KB | 2026-02-16 00:00:10
ZIP: PR260204_20260204.zip
Counter({'.zip': 1})
Arquivos: 1
- PR260204.zip | 8040.9 KB | 2026-02-16 01:10:10


Muito interessante observar que há dentro de cada arquivo compactado um arquivo compactado ( .zip ) também. Olhando então mais afundo:

In [ ]:
import zipfile
from pathlib import Path
from collections import Counter
from datetime import datetime
from io import BytesIO

for zp in zip_path:
    with zipfile.ZipFile(zp, "r") as z:
        infos = z.infolist()
        print(f"\nZIP EXTERNO: {zp.name}")
        
        exts = [Path(i.filename).suffix.lower() 
                for i in infos if not i.is_dir()]
        print("Extensões:", Counter(exts))
        print(f"Arquivos: {len(infos)}")

        for i in infos:
            dt = datetime(*i.date_time)
            print(f"- {i.filename} | {i.file_size/1024:.1f} KB | {dt}")

            # Se for um ZIP interno, abrir na memória
            if i.filename.lower().endswith(".zip"):
                print(f"\n  >>> Abrindo ZIP interno: {i.filename}")

                with z.open(i) as inner_file:
                    inner_bytes = BytesIO(inner_file.read())

                    with zipfile.ZipFile(inner_bytes, "r") as inner_zip:
                        inner_infos = inner_zip.infolist()
                        inner_exts = [
                            Path(j.filename).suffix.lower()
                            for j in inner_infos if not j.is_dir()
                        ]

                        print("  Extensões internas:", Counter(inner_exts))
                        print(f"  Arquivos internos: {len(inner_infos)}")

                        for j in inner_infos[:20]:
                            dt2 = datetime(*j.date_time)
                            print(f"   - {j.filename} | {j.file_size/1024:.1f} KB | {dt2}")



ZIP EXTERNO: PR200102_20200102.zip
Extensões: Counter({'.zip': 1})
Arquivos: 1
- PR200102.zip | 2417.4 KB | 2026-02-15 23:40:14

  >>> Abrindo ZIP interno: PR200102.zip
  Extensões internas: Counter({'.xml': 3})
  Arquivos internos: 3
   - BVBG.086.01_BV000328202001020328000001830098585.xml | 35566.1 KB | 2020-01-02 18:30:50
   - BVBG.086.01_BV000328202001020328000001900552975.xml | 35650.7 KB | 2020-01-02 19:01:34
   - BVBG.086.01_BV000328202001020328000001952035761.xml | 35658.0 KB | 2020-01-02 19:52:44

ZIP EXTERNO: PR220517_20220517.zip
Extensões: Counter({'.zip': 1})
Arquivos: 1
- PR220517.zip | 4915.4 KB | 2026-02-16 00:00:10

  >>> Abrindo ZIP interno: PR220517.zip
  Extensões internas: Counter({'.xml': 3})
  Arquivos internos: 3
   - BVBG.086.01_BV000328202205170328000001809111380.xml | 66746.0 KB | 2022-05-17 18:10:08
   - BVBG.086.01_BV000328202205170328000001858502601.xml | 66760.4 KB | 2022-05-17 18:59:38
   - BVBG.086.01_BV000328202205170328000001922141813.xml | 66760.4 K

Veja então que consistentemente cada arquivo compactado possui um único arquivo .zip que por sua vez possue 3 arquivos .xml dentro de si. Pela quantidade de arquivos (Estou rodando desde 01/01/2020) é inteligente começar a montar uma estratégia que não carregue (ou carregue o mínimo)desses arquivos na memória e salve apenas o nescessário.

In [5]:
zip_externo = zip_path[0]
with zipfile.ZipFile(zip_externo, "r") as z:
    inner_name = [n for n in z.namelist() if n.lower().endswith(".zip")][0]
    inner_bytes = BytesIO(z.read(inner_name))

with zipfile.ZipFile(inner_bytes, "r") as zi:
    xml_name = [n for n in zi.namelist() if n.lower().endswith(".xml")][0]
    with zi.open(xml_name) as f:
        print(f.read(3000).decode("utf-8", errors="replace"))


<?xml version="1.0" encoding="utf-8"?>
<Document xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="urn:bvmf.052.01.xsd bvmf.052.01.xsd" xmlns="urn:bvmf.052.01.xsd">
  <BizFileHdr>
    <Xchg>
      <BizGrpDesc>
        <Fr>
          <OrgId>
            <Id>
              <OrgId>
                <Othr>
                  <Id>BVMF</Id>
                  <Issr>40</Issr>
                  <SchmeNm>
                    <Prtry>39</Prtry>
                  </SchmeNm>
                </Othr>
              </OrgId>
            </Id>
          </OrgId>
        </Fr>
        <To>
          <OrgId>
            <Id>
              <OrgId>
                <Othr>
                  <Id>PUBLIC</Id>
                  <Issr>40</Issr>
                  <SchmeNm>
                    <Prtry>39</Prtry>
                  </SchmeNm>
                </Othr>
              </OrgId>
            </Id>
          </OrgId>
        </To>
        <BizGrp

In [9]:
import re
import xml.etree.ElementTree as ET
import pandas as pd

# ajuste conforme o padrão real do seu arquivo
DI_FUT_RE = re.compile(r"^DI1")  # ex.: DI1F26, DI1N26 etc.

def strip_ns(tag: str) -> str:
    return tag.split("}", 1)[-1] if "}" in tag else tag

def parse_di_futuro(xml_file):
    rows = []
    ctx = ET.iterparse(xml_file, events=("end",))

    for event, elem in ctx:
        if strip_ns(elem.tag) != "PricRpt":
            continue

        # 1) ticker
        tck = None
        for child in elem.iter():
            if strip_ns(child.tag) == "TckrSymb":
                tck = (child.text or "").strip()
                break

        # filtra cedo
        if not tck or not DI_FUT_RE.search(tck):
            elem.clear()
            continue

        # 2) trade date
        trade_dt = None
        for child in elem.iter():
            if strip_ns(child.tag) == "Dt":
                trade_dt = (child.text or "").strip()
                break

        # 3) aqui você extrai os campos de preço que existirem no seu XML:
        # procure por tags típicas como SttlmPric, LastPric, TradPric, etc.
        # (depende do schema bvmf.217.01.xsd)
        last_price = None
        sttl_price = None

        for child in elem.iter():
            tag = strip_ns(child.tag)
            if tag in ("LastPric", "LastPx"):
                last_price = (child.text or "").strip()
            elif tag in ("SttlmPric", "SttlmPx", "SttlmPrice"):
                sttl_price = (child.text or "").strip()

        rows.append({
            "trade_date": trade_dt,
            "ticker": tck,
            "last_price": last_price,
            "settlement_price": sttl_price,
        })

        # libera memória do nó já processado
        elem.clear()

    return pd.DataFrame(rows)


In [13]:
zip_externo = zip_path[0]
with zipfile.ZipFile(zip_externo, "r") as z:
    inner_name = [n for n in z.namelist() if n.lower().endswith(".zip")][0]
    inner_bytes = BytesIO(z.read(inner_name))

with zipfile.ZipFile(inner_bytes, "r") as zi:
    xml_name = [n for n in zi.namelist() if n.lower().endswith(".xml")][0]
    with zi.open(xml_name) as f:
        head = f.read(500)
        print(head.decode("utf-8", errors="replace"))
    
    with zi.open(xml_name) as f:
        data = f.read()  # só para diagnosticar 1x; depois evitamos
    print("ocorrências de <?xml:", data.count(b"<?xml"))
    print("últimos 300 bytes:\n", data[-300:].decode("utf-8", errors="replace"))



<?xml version="1.0" encoding="utf-8"?>
<Document xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="urn:bvmf.052.01.xsd bvmf.052.01.xsd" xmlns="urn:bvmf.052.01.xsd">
  <BizFileHdr>
    <Xchg>
      <BizGrpDesc>
        <Fr>
          <OrgId>
            <Id>
              <OrgId>
                <Othr>
                  <Id>BVMF</Id>
                  <Issr>40</Issr>
                  <SchmeNm>
                  
ocorrências de <?xml: 1
últimos 300 bytes:
       <AdjstdValCtrct Ccy="BRL">157.5</AdjstdValCtrct>
              <MaxTradLmt Ccy="BRL">44.1</MaxTradLmt>
              <MinTradLmt Ccy="BRL">39.9</MinTradLmt>
            </FinInstrmAttrbts>
          </PricRpt>
        </Document>
      </BizGrp>
    </Xchg>
  </BizFileHdr>
</Document>


In [15]:
zip_externo = zip_path[0]
with zipfile.ZipFile(zip_externo, "r") as z:
    inner_name = [n for n in z.namelist() if n.lower().endswith(".zip")][0]
    inner_bytes = BytesIO(z.read(inner_name))

with zipfile.ZipFile(inner_bytes, "r") as zi:
    xml_name = [n for n in zi.namelist() if n.lower().endswith(".xml")][0]

    raw = zi.read(xml_name)

    # últimos 50 bytes em hex
    print(raw[-50:].hex())

    # verifica se tem NUL
    print("tem NUL (\\x00)?", b"\x00" in raw[-200:])

    # verifica se tem bytes não-whitespace depois do fechamento
    end_tag = b"</Document>"
    pos = raw.rfind(end_tag)
    tail = raw[pos + len(end_tag):]
    print("tamanho do tail após </Document>:", len(tail))
    print("tail (hex):", tail[:80].hex())


42697a4772703e0d0a202020203c2f586368673e0d0a20203c2f42697a46696c654864723e0d0a3c2f446f63756d656e743e
tem NUL (\x00)? False
tamanho do tail após </Document>: 0
tail (hex): 


In [16]:
import zipfile
from io import BytesIO

with zipfile.ZipFile(zip_externo, "r") as z:
    inner_name = [n for n in z.namelist() if n.lower().endswith(".zip")][0]
    inner_bytes = BytesIO(z.read(inner_name))

with zipfile.ZipFile(inner_bytes, "r") as zi:
    for xml_name in [n for n in zi.namelist() if n.lower().endswith(".xml")]:
        raw = zi.read(xml_name)
        cnt = raw.count(b"<?xml")
        end_tag = b"</Document>"
        pos = raw.rfind(end_tag)
        tail = raw[pos + len(end_tag):] if pos != -1 else b""
        print(xml_name, "count<?xml=", cnt, "tail_len=", len(tail))


BVBG.086.01_BV000328202001020328000001830098585.xml count<?xml= 1 tail_len= 0
BVBG.086.01_BV000328202001020328000001900552975.xml count<?xml= 1 tail_len= 0
BVBG.086.01_BV000328202001020328000001952035761.xml count<?xml= 1 tail_len= 0


In [17]:
import xml.etree.ElementTree as ET
from collections import Counter

def strip_ns(tag: str) -> str:
    return tag.split("}", 1)[-1] if "}" in tag else tag

def iter_di1_tickers(xml_file):
    """
    Retorna gerador de tickers que começam com DI1.
    Não guarda o XML inteiro; vai limpando memória.
    """
    ctx = ET.iterparse(xml_file, events=("end",))
    for event, elem in ctx:
        if strip_ns(elem.tag) != "PricRpt":
            continue

        # acha o TckrSymb dentro desse PricRpt
        tck = None
        for node in elem.iter():
            if strip_ns(node.tag) == "TckrSymb":
                tck = (node.text or "").strip()
                break

        if tck and tck.startswith("DI1"):
            yield tck

        elem.clear()

zip_externo = zip_path[0]
with zipfile.ZipFile(zip_externo, "r") as z:
    inner_name = [n for n in z.namelist() if n.lower().endswith(".zip")][0]
    inner_bytes = BytesIO(z.read(inner_name))

with zipfile.ZipFile(inner_bytes, "r") as zi:
    xml_name = [n for n in zi.namelist() if n.lower().endswith(".xml")][0]

    with zi.open(xml_name) as f:
        ticks = list(iter_di1_tickers(f))

    print("qtde DI1:", len(ticks))
    print("exemplos:", ticks[:20])
    print("distintos:", len(set(ticks)))


qtde DI1: 37
exemplos: ['DI1G20', 'DI1U20', 'DI1N23', 'DI1J20', 'DI1F25', 'DI1X20', 'DI1V24', 'DI1J24', 'DI1F21', 'DI1F31', 'DI1V20', 'DI1N26', 'DI1F29', 'DI1N22', 'DI1N21', 'DI1Q20', 'DI1H20', 'DI1F28', 'DI1J23', 'DI1F24']
distintos: 37


In [ ]:
import re
from collections import defaultdict

def summarize_ticker_patterns(tickers):
    by_len = Counter(map(len, tickers))
    prefix = Counter(t[:3] for t in tickers)
    # captura "DI1" + 1 letra + 2 dígitos (padrão muito comum)
    re_basic = re.compile(r"^(DI1)([A-Z])(\d{2})$")
    ok, bad = [], []
    for t in set(tickers):
        m = re_basic.match(t)
        (ok if m else bad).append(t)

    return {
        "by_len": by_len,
        "prefix": prefix,
        "basic_ok": len(ok),
        "basic_bad": len(bad),
        "examples_ok": sorted(ok)[:15],
        "examples_bad": sorted(bad)[:15],
    }

stats = summarize_ticker_patterns(ticks)
print(stats)


{'by_len': Counter({6: 37}), 'prefix': Counter({'DI1': 37}), 'basic_ok': 37, 'basic_bad': 0, 'examples_ok': ['DI1F20', 'DI1F21', 'DI1F22', 'DI1F23', 'DI1F24', 'DI1F25', 'DI1F26', 'DI1F27', 'DI1F28', 'DI1F29', 'DI1F30', 'DI1F31', 'DI1G20', 'DI1H20', 'DI1J20'], 'examples_bad': []}


In [22]:
def iter_di1_records(xml_file):
    ctx = ET.iterparse(xml_file, events=("end",))
    for event, elem in ctx:
        if strip_ns(elem.tag) != "PricRpt":
            continue

        tck = None
        for node in elem.iter():
            if strip_ns(node.tag) == "TckrSymb":
                tck = (node.text or "").strip()
                break

        if not tck or not tck.startswith("DI1"):
            elem.clear()
            continue

        # trade date (primeiro <Dt> que aparece costuma ser a data do pregão)
        trade_dt = None
        for node in elem.iter():
            if strip_ns(node.tag) == "Dt":
                trade_dt = (node.text or "").strip()
                break

        # IMPORTANTÍSSIMO: essas tags variam por schema. Então pegamos por "candidatos".
        last_px = None
        sttl_px = None
        for node in elem.iter():
            tag = strip_ns(node.tag)
            if tag in ("LastPric", "LastPx"):
                last_px = (node.text or "").strip()
            elif tag in ("SttlmPric", "SttlmPx", "SttlmPrice"):
                sttl_px = (node.text or "").strip()

        yield {
            "trade_date": trade_dt,
            "ticker": tck,
            "last_price": last_px,
            "settlement_price": sttl_px,
        }

        elem.clear()

zip_externo = zip_path[0]
with zipfile.ZipFile(zip_externo, "r") as z:
    inner_name = [n for n in z.namelist() if n.lower().endswith(".zip")][0]
    inner_bytes = BytesIO(z.read(inner_name))

with zipfile.ZipFile(inner_bytes, "r") as zi:
    xml_name = [n for n in zi.namelist() if n.lower().endswith(".xml")][0]

    with zi.open(xml_name) as f:
        ticks_r = list(iter_di1_records(f))
print(*ticks_r, sep='\n')

{'trade_date': '2020-01-02', 'ticker': 'DI1G20', 'last_price': '4.41', 'settlement_price': None}
{'trade_date': '2020-01-02', 'ticker': 'DI1U20', 'last_price': None, 'settlement_price': None}
{'trade_date': '2020-01-02', 'ticker': 'DI1N23', 'last_price': '5.98', 'settlement_price': None}
{'trade_date': '2020-01-02', 'ticker': 'DI1J20', 'last_price': '4.336', 'settlement_price': None}
{'trade_date': '2020-01-02', 'ticker': 'DI1F25', 'last_price': '6.38', 'settlement_price': None}
{'trade_date': '2020-01-02', 'ticker': 'DI1X20', 'last_price': None, 'settlement_price': None}
{'trade_date': '2020-01-02', 'ticker': 'DI1V24', 'last_price': None, 'settlement_price': None}
{'trade_date': '2020-01-02', 'ticker': 'DI1J24', 'last_price': None, 'settlement_price': None}
{'trade_date': '2020-01-02', 'ticker': 'DI1F21', 'last_price': '4.52', 'settlement_price': None}
{'trade_date': '2020-01-02', 'ticker': 'DI1F31', 'last_price': None, 'settlement_price': None}
{'trade_date': '2020-01-02', 'ticker': 

In [23]:
def debug_tags_for_first_di1(xml_file, limit=1):
    ctx = ET.iterparse(xml_file, events=("end",))
    seen = 0
    for event, elem in ctx:
        if strip_ns(elem.tag) != "PricRpt":
            continue

        tck = None
        for node in elem.iter():
            if strip_ns(node.tag) == "TckrSymb":
                tck = (node.text or "").strip()
                break

        if tck and tck.startswith("DI1"):
            tags = Counter(strip_ns(n.tag) for n in elem.iter())
            print("ticker:", tck)
            print("top tags:", tags.most_common(40))
            seen += 1
            if seen >= limit:
                return
        elem.clear()
zip_externo = zip_path[0]
with zipfile.ZipFile(zip_externo, "r") as z:
    inner_name = [n for n in z.namelist() if n.lower().endswith(".zip")][0]
    inner_bytes = BytesIO(z.read(inner_name))

with zipfile.ZipFile(inner_bytes, "r") as zi:
    xml_name = [n for n in zi.namelist() if n.lower().endswith(".xml")][0]

    with zi.open(xml_name) as f:
        debug_tags_for_first_di1(f, limit=1)
        f.seek(0)
        ticks_r = list(iter_di1_records(f))

ticker: DI1G20
top tags: [('PricRpt', 1), ('TradDt', 1), ('Dt', 1), ('SctyId', 1), ('TckrSymb', 1), ('FinInstrmId', 1), ('OthrId', 1), ('Id', 1), ('Tp', 1), ('Prtry', 1), ('PlcOfListg', 1), ('MktIdrCd', 1), ('TradDtls', 1), ('TradQty', 1), ('FinInstrmAttrbts', 1), ('MktDataStrmId', 1), ('NtlFinVol', 1), ('IntlFinVol', 1), ('OpnIntrst', 1), ('FinInstrmQty', 1), ('BestBidPric', 1), ('BestAskPric', 1), ('FrstPric', 1), ('MinPric', 1), ('MaxPric', 1), ('TradAvrgPric', 1), ('LastPric', 1), ('RglrTxsQty', 1), ('RglrTraddCtrcts', 1), ('NtlRglrVol', 1), ('IntlRglrVol', 1), ('AdjstdQt', 1), ('AdjstdQtTax', 1), ('AdjstdQtStin', 1), ('PrvsAdjstdQt', 1), ('PrvsAdjstdQtTax', 1), ('PrvsAdjstdQtStin', 1), ('OscnPctg', 1), ('VartnPts', 1), ('AdjstdValCtrct', 1)]


trade_date, ticker, (e depois maturity)

AdjstdValCtrct (settlement/ajuste)

BestBidPric, BestAskPric (para construir mid e intervalos)

TradQty, OpnIntrst (liquidez)

LastPric (útil como diagnóstico)

Úteis para diagnósticos e microestrutura

TradAvrgPric, MinPric, MaxPric, FrstPric

contagens (RglrTxsQty, etc.)

OscnPctg, VartnPts (sanity checks)

## Calendário : *ANBIMA*


In [2]:
import pandas as pd
holidays = pd.read_parquet(r'C:\Users\Dell\OneDrive\Documentos\GitHub\ML-ETTJ26\data\calendars\02_trusted\ref\anbima_holidays.parquet')
bu_index = pd.read_parquet(r'C:\Users\Dell\OneDrive\Documentos\GitHub\ML-ETTJ26\data\calendars\02_trusted\ref\calendar_bd_index.parquet')

In [5]:
print(holidays.info(), bu_index.info(), sep="\n\n")

<class 'pandas.DataFrame'>
RangeIndex: 1263 entries, 0 to 1262
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   cal_id            1263 non-null   str   
 1   date              1263 non-null   object
 2   holiday_name      1263 non-null   str   
 3   weekday           1263 non-null   int32 
 4   ingestion_ts_utc  1263 non-null   str   
 5   source_file_hash  1263 non-null   str   
 6   pipeline_run_id   1263 non-null   str   
dtypes: int32(1), object(1), str(5)
memory usage: 214.4+ KB
<class 'pandas.DataFrame'>
RangeIndex: 36159 entries, 0 to 36158
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   cal_id            36159 non-null  str   
 1   date              36159 non-null  object
 2   weekday           36159 non-null  int32 
 3   is_business_day   36159 non-null  bool  
 4   bd_index          36159 non-null  int64 
 5   holiday

In [7]:
bu_index.head(10)

,cal_id,date,weekday,is_business_day,bd_index,holiday_name,ingestion_ts_utc,source_file_hash,pipeline_run_id
0,BR_ANBIMA,2001-01-01,0,False,0,Confraternização Universal,2026-02-21T13:54:11+00:00,bff506cbcab013530f5b522ad8b0b2cec0c967dce9b245...,local
1,BR_ANBIMA,2001-01-02,1,True,1,NaN,2026-02-21T13:54:11+00:00,bff506cbcab013530f5b522ad8b0b2cec0c967dce9b245...,local
2,BR_ANBIMA,2001-01-03,2,True,2,NaN,2026-02-21T13:54:11+00:00,bff506cbcab013530f5b522ad8b0b2cec0c967dce9b245...,local
3,BR_ANBIMA,2001-01-04,3,True,3,NaN,2026-02-21T13:54:11+00:00,bff506cbcab013530f5b522ad8b0b2cec0c967dce9b245...,local
4,BR_ANBIMA,2001-01-05,4,True,4,NaN,2026-02-21T13:54:11+00:00,bff506cbcab013530f5b522ad8b0b2cec0c967dce9b245...,local
5,BR_ANBIMA,2001-01-06,5,False,4,NaN,2026-02-21T13:54:11+00:00,bff506cbcab013530f5b522ad8b0b2cec0c967dce9b245...,local
6,BR_ANBIMA,2001-01-07,6,False,4,NaN,2026-02-21T13:54:11+00:00,bff506cbcab013530f5b522ad8b0b2cec0c967dce9b245...,local
7,BR_ANBIMA,2001-01-08,0,True,5,NaN,2026-02-21T13:54:11+00:00,bff506cbcab013530f5b522ad8b0b2cec0c967dce9b245...,local
8,BR_ANBIMA,2001-01-09,1,True,6,NaN,2026-02-21T13:54:11+00:00,bff506cbcab013530f5b522ad8b0b2cec0c967dce9b245...,local
9,BR_ANBIMA,2001-01-10,2,True,7,NaN,2026-02-21T13:54:11+00:00,bff506cbcab013530f5b522ad8b0b2cec0c967dce9b245...,local
